## Install Requirements

In [1]:
pip install datasets jiwer transformers colorednoise pyctcdecode https://github.com/kpu/kenlm/archive/master.zip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 550 kB 3.2 MB/s 
     |████████████████████████████████| 441 kB 5.1 MB/s 
     |████████████████████████████████| 5.3 MB 67.9 MB/s 
     |████████████████████████████████| 45 kB 3.6 MB/s 
     |████████████████████████████████| 115 kB 56.2 MB/s 
     |████████████████████████████████| 212 kB 73.7 MB/s 
     |████████████████████████████████| 163 kB 68.1 MB/s 
     |████████████████████████████████| 127 kB 73.0 MB/s 
     |████████████████████████████████| 1.4 MB 45.7 MB/s 
     |████████████████████████████████| 2.2 MB 47.7 MB/s 
     |████████████████████████████████| 7.6 MB 58.0 MB/s 
     |████████████████████████████████| 395 kB 72.6 MB/s 
  Created wheel for kenlm: filename=kenlm-0.0.0-cp37-cp37m-linux_x86_64.whl size=2372798 sha256=0a0df5645a7fbc8a7e2c8edcfe229e3cec861098547ec9421de183d79ae0498a
  Stored in directory: /tmp/pip-ephem-wheel-cach

## Imports

In [2]:
import pandas as pd
import numpy as np
import os
import kenlm
from transformers import Wav2Vec2ProcessorWithLM, Wav2Vec2ForCTC, Wav2Vec2Tokenizer, Wav2Vec2Processor, AutoModelForCTC, AutoProcessor
from datasets import load_dataset
from jiwer import wer
import librosa
import nltk
import tarfile
import torch
import urllib.request
import soundfile as sf

nltk.download('punkt')

import audio_preprocess

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## Required Functions

In [3]:
def download_and_extract_dataset_from_url(url, datasets_path):
    """
    downloads and extracts dataset from url into datasets_path/
    """
    temp = os.path.join(datasets_path, url.split('/')[-1])
    urllib.request.urlretrieve(url, temp)
    file = tarfile.open(temp)
    file.extractall(datasets_path)
    file.close()
    os.remove(temp)

In [4]:
def map_to_ground_truth(batch):
    """
    inserts ground truth in dataset
    """
    transcription_file_path = batch['audio']['path'][:-10] + '.trans.txt'
    f = open(transcription_file_path, 'r')
    lines= str.splitlines(f.read())
    txt=lines[int(batch['audio']['path'][-7:-5])].split(' ', 1)[1]
    batch['txt'] = txt
    return batch

In [5]:
def load_wav2vec_model(process_path: str):
    """
    load and return wav2vec tokenizer and model from huggingface
    """
    model = AutoModelForCTC.from_pretrained(process_path).to("cuda")
    processor = AutoProcessor.from_pretrained(process_path)
    return processor, model

In [6]:
def map_to_pred(batch):
    """
    predicts transcription
    """
    #tokenize
    inputs = processor(batch["audio"]["array"], sampling_rate=16_000, return_tensors="pt")
    inputs = {k: v.to("cuda") for k,v in inputs.items()}
    #take logits
    with torch.no_grad():
        logits = model(**inputs).logits

    transcription = processor.batch_decode(logits.cpu().numpy()).text[0]
    batch["transcription"] = transcription
    return batch
    

In [7]:
def add_noise_to_dataset(batch):
    """
    adds noise to dataset
    """
    batch['audio']['array'] = audio_preprocess.add_noise(batch['audio']['array'])
    return batch

In [8]:
def downsample_dataset(batch):
    """
    downsamples dataset
    """
    batch['audio']['array'] = audio_preprocess.down_sample(batch['audio']['array'])
    return batch

In [9]:
def format_wer(text, transcription, decimal=1):
  return round(100 * wer(text, transcription), decimal)

## wav2vec 2.0 on Noisy Data

In [10]:
# set paths
datasets_path = os.path.join(os.getcwd(), 'datasets') 
# create folders if they do not already exist
if not os.path.exists(datasets_path): os.makedirs(datasets_path)
# set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [11]:
download_and_extract_dataset_from_url('https://www.openslr.org/resources/12/test-clean.tar.gz', datasets_path)
download_and_extract_dataset_from_url('https://www.openslr.org/resources/12/test-other.tar.gz', datasets_path)

In [12]:
# load extracted ls data as dataset
librispeech_clean = load_dataset("datasets/LibriSpeech", "clean", split='test')
librispeech_other = load_dataset("datasets/LibriSpeech", "other", split='test')

Resolving data files:   0%|          | 0/5736 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/177 [00:00<?, ?it/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset audiofolder downloaded and prepared to /root/.cache/huggingface/datasets/audiofolder/LibriSpeech-f21c4936e377859a/0.0.0/6cbdd16f8688354c63b4e2a36e1585d05de285023ee6443ffd71c4182055c0fc. Subsequent calls will reuse this data.


Resolving data files:   0%|          | 0/5736 [00:00<?, ?it/s]

In [13]:
librispeech_clean = librispeech_clean.map(map_to_ground_truth)
librispeech_other = librispeech_other.map(map_to_ground_truth)

  0%|          | 0/5559 [00:00<?, ?ex/s]

In [14]:
librispeech_clean_noisy = librispeech_clean.map(add_noise_to_dataset)
librispeech_clean_downsampled = librispeech_clean.map(downsample_dataset)

  0%|          | 0/5559 [00:00<?, ?ex/s]

  0%|          | 0/5559 [00:00<?, ?ex/s]

In [15]:
processor, model = load_wav2vec_model("patrickvonplaten/wav2vec2-base-960h-4-gram")

Downloading:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at patrickvonplaten/wav2vec2-base-960h-4-gram and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Downloading:   0%|          | 0.00/263 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/291 [00:00<?, ?B/s]

...

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/198 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.12G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/78.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.74M [00:00<?, ?B/s]

In [16]:
clean_result = librispeech_clean.map(map_to_pred, remove_columns=["audio"])
other_result = librispeech_other.map(map_to_pred, remove_columns=["audio"])
clean_noisy_result = librispeech_clean_noisy.map(map_to_pred, remove_columns=["audio"])
clean_downsampled_result = librispeech_clean_downsampled.map(map_to_pred, remove_columns=["audio"])

  0%|          | 0/5559 [00:00<?, ?ex/s]

  0%|          | 0/5559 [00:00<?, ?ex/s]

  0%|          | 0/5559 [00:00<?, ?ex/s]

  0%|          | 0/5559 [00:00<?, ?ex/s]

In [17]:
print('WER: wav2vec2-base-960h-4-gram, ls-test-clean:', format_wer(clean_result["txt"], clean_result["transcription"]), '%.')
print('WER: wav2vec2-base-960h-4-gram, ls-test-other:', format_wer(other_result["txt"], other_result["transcription"]), '%.')
print('WER: wav2vec2-base-960h-4-gram, ls-test-clean, noisy:', format_wer(clean_noisy_result["txt"], clean_noisy_result["transcription"]), '%.')
print('WER: wav2vec2-base-960h-4-gram, ls-test-clean, downsampled:', format_wer(clean_downsampled_result["txt"], clean_downsampled_result["transcription"]), '%.')

WER: wav2vec2-base-960h-4-gram, ls-test-clean: 4.9 %.
WER: wav2vec2-base-960h-4-gram, ls-test-other: 4.9 %.
WER: wav2vec2-base-960h-4-gram, ls-test-clean, noisy: 13.1 %.
WER: wav2vec2-base-960h-4-gram, ls-test-clean, downsampled: 6.6 %.
